In [ ]:
# Hyperparameters for training a neural network on the MNIST dataset

# Learning rate controls the step size of gradient descent, determining how much to update the weights in each iteration.
learning_rate = 0.02

# Number of epochs specifies how many complete passes through the training dataset will be performed.
num_epochs = 100

# Batch size defines the number of images processed in each mini-batch during training.
batch_size = 32

# Percentage of the dataset to be used for training, allowing for experimentation with smaller subsets of data.
train_data_percentage = 0.0025
test_data_percentage = 0.1

# Input size corresponds to the dimensions of the MNIST images, flattened from 28x28 pixels.
input_size = 28 * 28  # MNIST image size

# Number of output classes, representing the digits 0 through 9 in the MNIST dataset.
num_classes = 10  # Number of classes in MNIST

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, random_split

# Dataset characteristics
We use the MNIST digit classification data set for this assignment.
A total of 60K images for training and 10K images for testing are available. But we only use a small percentage of them. Images are 28 x 28 pixels.

In [ ]:
# Load MNIST datasets, and create pytorch data loader to read data in mini-batches
def get_data_loaders(train_data_percentage, test_data_percentage, batch_size, transform):
    # Load the entire MNIST dataset
    # For train and test data points we sometimes use different transforms.
    # This becomes handy in the last task (data augmentation)
    full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_transform = transforms.Compose([transforms.ToTensor()])
    full_test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=test_transform)

    # Calculate the size based on the percentage
    train_size = int(train_data_percentage * len(full_train_dataset))  # percentage of training data
    test_size = int(test_data_percentage * len(full_test_dataset))    # percentage of test data
    train_remainder = len(full_train_dataset) - train_size
    test_remainder = len(full_test_dataset) - test_size

    # Split the dataset into the percentage specified and the remaining
    train_dataset, _ = random_split(full_train_dataset, [train_size, train_remainder])
    test_dataset, _ = random_split(full_test_dataset, [test_size, test_remainder])

    # Create DataLoaders for batching and shuffling
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [ ]:
# Define transformation (convert to tensor)
transform = transforms.Compose([transforms.ToTensor()])
train_loader, test_loader = get_data_loaders(train_data_percentage, test_data_percentage,batch_size, transform)

In [ ]:
# Get first batch of images and labels
train_image_batch, classe_set = next(iter(train_loader))

print(f'train_loader contains {len(train_loader)} batches of data.')
print(f'train_image_batch has shape {train_image_batch.shape},')
print('where 64 is the number of images in a batch, 1 is the number of image channels (1 for grayscale image),\
 28X28 stands for WxH (width and height of a single image).')

# Visualization of the dataset

In [ ]:
def show_gray_digits(image_set, row=2, col=3):
    # Here we visualize some of the data points in the data set.
    # Create a large figure, to be filled with multiple subplots.

    # Since image_set is a tensor variable, we transform it to a numpy type variable.
    image_set = image_set.cpu().detach().numpy()

    for i in range(row*col):
      # define subplot
      plt.subplot(row, col, i+1)
      # plot raw pixel data
      plt.imshow(image_set[i,0], cmap=plt.get_cmap('gray'))
    # show the figure
    plt.show()

In [ ]:
# display images and their corresponding labels.
show_gray_digits(train_image_batch, 2, 3)
print(classe_set[:6])

del train_image_batch, classe_set, train_loader, test_loader, transform

# Linear SVM for MNIST classification 


## Recap

- **Accuracy Computation**: 
  - Calculated the accuracy on both the train and test sets after each epoch during training.
  - Plotted the accuracies as a function of the number of epochs for both sets.

- **Hinge Loss Computation**: 
  - Calculated the hinge loss on both the train and test sets after each epoch.
  - Plotted the hinge loss values as a function of the number of epochs.

- **Final Epoch Results**:
  - Recorded the loss values and accuracies for the final epoch of training for both the train and test sets.

- **Overfitting Analysis**:
  - Analyzed whether the model exhibited significant overfitting, considering other potential factors contributing to the model's performance.


In [ ]:
# Function to plot train/test loss and accuracy on separate subplots
def plot_eval_results(train_loss, test_loss, train_acc, test_acc):
    """
    Plots the training and testing loss/accuracy over the number of epochs.

    Parameters:
    - train_loss: list or array, the training loss values over epochs.
    - test_loss: list or array, the testing loss values over epochs.
    - train_acc: list or array, the training accuracy values over epochs.
    - test_acc: list or array, the testing accuracy values over epochs.
    """


    # Create subplots (1 row, 2 columns) for loss and accuracy
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))  # figsize sets the overall plot size

    # Plot Loss on the first subplot
    ax1.plot(range(len(train_loss)), train_loss, label="Training Loss", color='blue')
    ax1.plot(range(len(test_loss)), test_loss, label="Testing Loss", color='red')

    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.set_title("Train vs Test Loss over epochs")


    # Plot Accuracy on the second subplot
    ax2.plot(range(len(train_acc)), train_acc, label="Training Accuracy", color="blue")
    ax2.plot(range(len(test_acc)), test_acc, label="Testing Accuracy", color="red")


    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy')
    ax2.set_title("Train vs Test Accuracy over epochs")

    # Adjust layout to avoid subplot overlap
    plt.tight_layout()
    ax1.legend(loc="lower right")
    ax2.legend(loc="lower right")




    # Display the plots
    plt.show()

In [ ]:
# Define a linear SVM model
class LinearSVM(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LinearSVM, self).__init__()
        self.fc = nn.Linear(input_size, num_classes, bias=True)

    def forward(self, x):
        # Flatten the image
        x= x.view(-1, input_size)
        return self.fc(x)

In [ ]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_loader, test_loader = get_data_loaders(train_data_percentage, test_data_percentage, batch_size, transform)

In [ ]:
# Return model loss and accuracy with the provided criterion and data_loader.
def eval(model, data_loader, criterion=None):
  model.eval()
  correct = 0
  total = 0
  loss_batches = []

  # Switch to evaluation mode and turn off gradient calculation
  # since parameters are not updated during testing.
  with torch.no_grad():
      for images_batch, labels_batch in data_loader:
          outputs = model(images_batch)
          # The predicted label is the output with the highest activation.
          _, predicted = torch.max(outputs.data, 1)
          total += labels_batch.size(0)
          correct += (predicted == labels_batch).sum().item()

          # Use provided criterion to calculate the loss for the mini batch
          # Append the mini-batch loss to loss_batches array
          batch_loss = criterion(outputs, labels_batch)
          loss_batches.append(batch_loss.item())

  accuracy = 100 * correct / total
  loss = np.mean(loss_batches)

  model.train()

  return accuracy, loss

In [ ]:
# Initialize the model, loss function, and optimizer
model = LinearSVM(input_size, num_classes)
criterion = nn.MultiMarginLoss()  # A Multi-class version of Hinge loss
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


train_loss_epochs = []
test_loss_epochs = []
train_accuracy_epochs = []
test_accuracy_epochs = []

# Training the model
for epoch in range(num_epochs):
    for i, (images_batch, labels_batch) in enumerate(train_loader):
        optimizer.zero_grad() # Clear the gradients
        outputs = model(images_batch) # Forward pass
        loss = criterion(outputs, labels_batch) # Calculate loss
        loss.backward() # Backward pass
        optimizer.step() # Update weights

    # Obtain train/test loss values and accuracies after each epoch
    train_accuracy, train_loss = eval(model, train_loader, criterion)
    test_accuracy, test_loss = eval(model, test_loader, criterion)

    print(f'Epoch {epoch+1:02d} - Train loss: {train_loss:.6f}, Train accuracy: {train_accuracy:.2f}%')
    print(f'         - Test loss: {test_loss:.6f}, Test accuracy: {test_accuracy:.2f}%')
    print("-------------------------------------------------------------")

    train_loss_epochs.append(train_loss)
    test_loss_epochs.append(test_loss)
    train_accuracy_epochs.append(train_accuracy)
    test_accuracy_epochs.append(test_accuracy)

In [ ]:
# Plotting
plot_eval_results(train_loss_epochs, test_loss_epochs, train_accuracy_epochs, test_accuracy_epochs)

## Part C 
## Weight Decay Experiment

- **Weight Decay Setup**:
  - Set weight decay to values of **0.1**, **1**, and **10** when defining the SGD optimizer.
  - Explored the impact of weight decay on regularization and model performance.

- **Loss and Accuracy Computation**:
  - Plotted the train and test losses and accuracies as a function of epochs for each weight decay setting.

- **Final Epoch Results**:
  - Recorded the loss and accuracy for both the train and test sets at the final epoch for each weight decay value.

- **Analysis**:
  - Analyzed whether weight decay improved the model's performance and whether it helped in reducing overfitting or underfitting. Justified the results based on the findings.



In [ ]:
for weight_decay in [0.1, 1, 10]:

    linear_svm = LinearSVM(input_size, num_classes)
    optimizer = torch.optim.SGD(linear_svm.parameters(), lr=learning_rate, weight_decay = weight_decay)

    train_loss_epochs = []
    test_loss_epochs = []
    train_accuracy_epochs = []
    test_accuracy_epochs = []

    for epoch in range(num_epochs):
      for images_batch, labels_batch in train_loader:
          optimizer.zero_grad() # Clear the gradients
          outputs = linear_svm(images_batch) # Forward pass
          loss = criterion(outputs, labels_batch) # Calculate loss
          loss.backward() # Backward pass
          optimizer.step() # Update weights


      train_accuracy, train_loss = eval(linear_svm, train_loader, criterion)
      test_accuracy, test_loss = eval(linear_svm, test_loader, criterion)

      train_loss_epochs.append(train_loss)
      test_loss_epochs.append(test_loss)

      train_accuracy_epochs.append(train_accuracy)
      test_accuracy_epochs.append(test_accuracy)



    # Plot accuracies
    print(f'For weight decay value {weight_decay}:')
    plot_eval_results(train_loss_epochs, test_loss_epochs, train_accuracy_epochs, test_accuracy_epochs)
    print(f"Final Train Loss: {train_loss}, Final Test Loss: {test_loss}")
    print(f"Final Train Accuracy: {train_accuracy}, Final Test Accuracy: {test_accuracy}")

# Logistic Regression for MNIST classification.
Here we implement a logistic regression model for the same MNIST classfication problem.


## Cross-Entropy Loss and Accuracy Computation

- **Accuracy Computation**:
  - Calculated the accuracy on both the train and test sets after each epoch during training.
  - Plotted the accuracies for both sets as a function of the number of epochs.

- **Cross-Entropy Loss Computation**:
  - Calculated the cross-entropy loss on both the train and test sets after each epoch.
  - Plotted the loss values for both sets as a function of the number of epochs.

- **Final Epoch Results**:
  - Recorded the loss values and accuracies for both the train and test sets at the final epoch of training.

- **Overfitting Analysis**:
  - Analyzed whether the model exhibited significant overfitting or if other factors (such as model architecture or training parameters) might explain any mediocre performance.



## Part C 

Comparing the results with SVM model

In [ ]:
# Logistic Regression model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
      super(LogisticRegression, self).__init__()
      self.fc = nn.Linear(input_size, num_classes, bias=True)


    def forward(self, x):
        # Flatten the image
        x= x.view(-1, input_size)
        return self.fc(x)

In [ ]:
# Model initialization
logistic_model = LogisticRegression(input_size, num_classes)
criterion = nn.CrossEntropyLoss()  # A Multi-class version of Hinge loss
optimizer = optim.SGD(logistic_model.parameters(), lr=learning_rate)

total_params = sum(p.numel() for p in logistic_model.parameters())
print(f"Number of parameters in the model: {total_params}")


# Train logistic regression model for MNIST classification

train_loss_epochs = []
test_loss_epochs = []
train_accuracy_epochs = []
test_accuracy_epochs = []


for epoch in range(num_epochs):
  for image, true_label_batch in train_loader:
    optimizer.zero_grad()
    outputs = logistic_model(image)
    loss = criterion(outputs, true_label_batch)

    loss.backward()
    optimizer.step()

  train_acc, train_loss = eval(logistic_model, train_loader, criterion)
  test_acc, test_loss = eval(logistic_model, test_loader, criterion)


  print(f'Epoch {epoch+1:02d} - Train loss: {train_loss:.6f}, Train accuracy: {train_acc:.2f}%')
  print(f'         - Test loss: {test_loss:.6f}, Test accuracy: {test_acc:.2f}%')
  print("-------------------------------------------------------------")

  train_loss_epochs.append(train_loss)
  test_loss_epochs.append(test_loss)

  train_accuracy_epochs.append(train_acc)
  test_accuracy_epochs.append(test_acc)

In [ ]:
# Plot the loss values and accuracies for train/test
plot_eval_results(train_loss_epochs, test_loss_epochs, train_accuracy_epochs, test_accuracy_epochs)

#  Non-linearity 



## Adding Hidden Layer and ReLU Activation

- **Model Modifications**:
  - Added a hidden layer with **5000 neurons** and a **ReLU activation** to both the **Logistic Regression** and **SVM** models.
  - This was done for both Task 1 (accuracy computation) and Task 2 (loss computation).

- **Train and Test Loss**:
  - Plotted the **train loss** and **test loss** for both models (Logistic Regression and SVM) after each epoch.

- **Train and Test Accuracies**:
  - Plotted the **train accuracies** and **test accuracies** for both models (Logistic Regression and SVM) as a function of the number of epochs.

- **Final Epoch Results**:
  - Reported the **loss** and **accuracy** for both the **train** and **test** sets at the final epoch for both models.


## Part B

## Comparison of Results: Modified vs. Linear Models

- **Model Comparison**:
  - Compared the performance of the **modified models** (with a hidden layer of 5000 neurons and a ReLU activation) against the **linear models** (without weight decay).
  - The comparison was based on metrics like **accuracy**, **loss**, and **overfitting** tendencies.

- **Performance Analysis**:
  - Evaluated which approach (linear model vs. modified model) produced **better results** in terms of accuracy and loss on both the **train** and **test** sets.
  - Plotted the corresponding **train and test losses** and **accuracies** for both approaches.

- **Overfitting Analysis**:
  - Analyzed the tendency of each model to **overfit** based on the performance gap between the train and test sets.
  - Discussed which model is **more prone to overfitting**, taking into account factors such as model complexity (with hidden layers vs. linear) and the absence of weight decay in the linear model.

- **Findings**:
  - Justified which approach works better based on the observed metrics.
  - Provided an explanation for the **overfitting behavior** of each model, with insights into why the modified model (with the hidden layer and ReLU) may be more prone to overfitting due to its increased complexity compared to the simpler linear model.


In [ ]:
# Both the SVM and Logistic Regression models we have in Task 1 and 2
# can be changed to the ModifiedModel below.

# Modified model with added neurons and relu layer
class ModifiedModel(nn.Module):
    def __init__(self, input_size, num_classes):
      super(ModifiedModel, self).__init__()
      self.hidden_size = 5000
      self.fc1 = nn.Linear(input_size, self.hidden_size) # input maps to 5000 neurons
      self.fc2 = nn.Linear(self.hidden_size, num_classes) # 5000 mapped to our 10 digit for classification
      self.relu = nn.ReLU() # activation function



    def forward(self, x):
        # Flatten the image
        x = x.view(-1, input_size)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Run the ModifiedModel with Hinge Loss (SVM)
# Model initialization

modified_SVM = ModifiedModel(input_size, num_classes)
criterion = nn.MultiMarginLoss()
optimizer = optim.SGD(modified_SVM.parameters(), lr= learning_rate)

train_loss_epochs = []
test_loss_epochs = []
train_accuracy_epochs = []
test_accuracy_epochs = []


for epoch in range(num_epochs):
  for image, true_label_batch in train_loader:
    optimizer.zero_grad()
    outputs = modified_SVM(image)
    loss = criterion(outputs, true_label_batch)

    loss.backward()
    optimizer.step()

  train_acc, train_loss = eval(modified_SVM, train_loader, criterion)
  test_acc, test_loss = eval(modified_SVM, test_loader, criterion)


  print(f'Epoch {epoch+1:02d} - Train loss: {train_loss:.6f}, Train accuracy: {train_acc:.2f}%')
  print(f'         - Test loss: {test_loss:.6f}, Test accuracy: {test_acc:.2f}%')
  print("-------------------------------------------------------------")

  train_loss_epochs.append(train_loss)
  test_loss_epochs.append(test_loss)

  train_accuracy_epochs.append(train_acc)
  test_accuracy_epochs.append(test_acc)





In [ ]:
# Plot the loss values and accuracies for train/test
plot_eval_results(train_loss_epochs, test_loss_epochs, train_accuracy_epochs, test_accuracy_epochs)

In [ ]:
# Run the ModifiedModel with Cross Entropy Loss (Logistic Regression)
# Model initialization

modified_logistic_regression_model = ModifiedModel(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(modified_logistic_regression_model.parameters(), lr= learning_rate)

train_loss_epochs = []
test_loss_epochs = []
train_accuracy_epochs = []
test_accuracy_epochs = []

for epoch in range(num_epochs):
  for image, true_label_batch in train_loader:
    optimizer.zero_grad()
    outputs = modified_logistic_regression_model(image)
    loss = criterion(outputs, true_label_batch)
    loss.backward()
    optimizer.step()

  train_acc, train_loss = eval(modified_logistic_regression_model, train_loader, criterion)
  test_acc, test_loss = eval(modified_logistic_regression_model, test_loader, criterion)


  print(f'Epoch {epoch+1:02d} - Train loss: {train_loss:.6f}, Train accuracy: {train_acc:.2f}%')
  print(f'         - Test loss: {test_loss:.6f}, Test accuracy: {test_acc:.2f}%')
  print("-------------------------------------------------------------")

  train_loss_epochs.append(train_loss)
  test_loss_epochs.append(test_loss)

  train_accuracy_epochs.append(train_acc)
  test_accuracy_epochs.append(test_acc)

In [ ]:
# Plot the loss values and accuracies for train/test
plot_eval_results(train_loss_epochs, test_loss_epochs, train_accuracy_epochs, test_accuracy_epochs)

# Data Augmentation 

## Data Augmentation Experiment

### Augmentation Approaches Tested

To improve the model’s performance, several alternative augmentation techniques were explored using PyTorch’s **transforms** module. The methods tested included:

- **Random Rotation**: Rotating images by a random angle to help the model learn rotational invariance.
- **Random Horizontal Flip**: Flipping images horizontally to simulate different viewing angles and increase dataset variability.
- **Random Crop**: Applying random crops to images to help the model focus on different parts of the image and improve its ability to generalize.
- **Color Jitter**: Adjusting the brightness, contrast, saturation, and hue of the images to simulate various lighting conditions.

After applying the new augmentation methods, the model was retrained, and the **train and test accuracies** were plotted for each epoch. The results were compared with the baseline test accuracy from Task 3.

### Results and Analysis

- The final test accuracy, train/test accuracy plots, and loss values were recorded and compared across epochs.
- The data augmentation technique that resulted in the **best improvement** (greater than 1% increase in test accuracy compared to Task 3) was identified.
- The reason for the selected augmentation technique’s effectiveness was explained, demonstrating how it contributed to better model generalization.

Overall, this task showed how the use of data augmentation helped improve the model's robustness, allowing it to handle variations in the input images and generalize better on the test set.






In [ ]:
# TRANSFOMATION
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.1),
    transforms.ToTensor()
])
train_loader_2, test_loader_2 = get_data_loaders(train_data_percentage, test_data_percentage, batch_size, transform)

In [ ]:
# Run the ModifiedModel with Hinge Loss (SVM)
# Model initialization

model = ModifiedModel(input_size, num_classes)
criterion = nn.MultiMarginLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)


train_loss_epochs = []
test_loss_epochs = []
train_accuracy_epochs = []
test_accuracy_epochs = []

for epoch in range(num_epochs):
  for image, true_label_batch in train_loader_2:
    optimizer.zero_grad()
    outputs = model(image)
    loss = criterion(outputs, true_label_batch)

    loss.backward()
    optimizer.step()

  train_acc, train_loss = eval(model, train_loader_2, criterion)
  test_acc, test_loss = eval(model, test_loader_2, criterion)

  train_loss_epochs.append(train_loss)
  test_loss_epochs.append(test_loss)

  train_accuracy_epochs.append(train_acc)
  test_accuracy_epochs.append(test_acc)

  print(f'Epoch {epoch+1:02d} - Train loss: {train_loss:.6f}, Train accuracy: {train_acc:.2f}%')
  print(f'         - Test loss: {test_loss:.6f}, Test accuracy: {test_acc:.2f}%')
  print("-------------------------------------------------------------")

In [ ]:
# Plot the loss values and accuracies for train/test
plot_eval_results(train_loss_epochs, test_loss_epochs, train_accuracy_epochs, test_accuracy_epochs)